In [1]:
from util import print_log, validate_model, sparse_validate_model

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15,30)

## Read raw data as lines

In [3]:
raw_train = pd.DataFrame([line for line in open('../data/classification_train.tsv',encoding='utf8')],columns=['line'])

In [4]:
raw_test = pd.DataFrame([line for line in open('../data/classification_blind_set_corrected.tsv',encoding='utf8')],columns=['line'])

## Extract category and brand from raw data

In [5]:
train = raw_train.line.str.extract(r'(.*)\t(\d+)\t(\d+)$',expand=True)
train.columns = ['product_title', 'brand_id', 'category_id']
train = train.dropna()
train.loc[:, ['brand_id', 'category_id']] = train.loc[:, ['brand_id', 'category_id']].astype(int)

In [6]:
test = raw_test.line.str.extract(r'(.*)\t(-?\d+)$',expand=True)
test.columns = ['product_title', 'category_id']
test = test.dropna()
test.loc[:, ['category_id']] = test.loc[:, ['category_id']].astype(int)

In [7]:
class Tokenizer2(object):
    def __init__(self):
        self.tokenizer = word_tokenize
        self.stop_words = set(
        ['is', 'of', 'it', 'at', 'on', 'and', 'as', 'the', 'to', 'are', 'this', 'that', 'be', 'in',
          'an', 'or','any', 'all', 'am','you','we', '__NUMBER__', '__SERIAL__'])

    def __call__(self, text):
        text = text.lower()
        # replace special characters
        text = re.sub(r'[^a-z0-9\s/\\_\t,\-]', '', text,flags=re.IGNORECASE)
        text = re.sub(r'[/\\_\t,-]', ' ', text,flags=re.IGNORECASE)
        # replace numbers to reduce number of features
        text = re.sub(r'\b[0-9]+\b', ' __NUMBER__ ', text) 
        # replace possible product/serial numbers
        text = re.sub(r'\b\w*\d+\w*\d?\b', ' __SERIAL__ ', text)
                
        tokens = [w for w in self.tokenizer(text) if (w not in self.stop_words and len(w)>1)]
        # only return first and last two tokens
        
        tokens = tokens if len(tokens) <5 else tokens[:3] + tokens[-2:]
        
        token_dict = {}
        
        for i,token in enumerate(tokens):
            token_dict['word_'+str(i)] = token
        
        return token_dict

In [8]:
! wc -l ../data/classification_train.tsv

1000000 ../data/classification_train.tsv


In [9]:
! wc -l ../data/classification_blind_set.tsv

wc: ../data/classification_blind_set.tsv: No such file or directory


In [10]:
train.shape, test.shape

((999996, 3), (619240, 2))

# missed rows

In [11]:
1000000 - train.shape[0], 619243 -  test.shape[0]

(4, 3)

In [12]:
pd.options.display.max_colwidth = 900

In [13]:
raw_train[~raw_train.line.str.contains(r'(.*)\t(\d+)\t(\d+)$')]

/opt/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


,line
218278,title\tbid\tcid\n
246806,""" 2 Pack Panasonic Compatible KX-FA83 KXFA83 Laser Toner Cartridge, 2,500 Pa\tUnknown\tcomputers & accessories > cables & accessories > printer ink & toner > laser printer toner\n"
458263,"""Brocade VDX 6720 - switch - 16 ports - rack-mountable\tBrocade Communication Systems\tcomputers & accessories > networking products > switches\n"
575503,"""This hub is built strong and ideal for industrial environments. With the StarTech.com ST4200USBM 4-port hub you can wall-mount or install onto a DIN rail for convenient access to the ports. This four port industrial hub can be bus powered or self powered with a three wire terminal block connector 7-24V . Plug in your most demanding next-generation peripherals and still enjoy data transfer speed\tStarTech\tcomputers & accessories > networking products > hubs\n"


## exploring category and brand cardinalities

In [14]:
all_data = pd.concat([train, test])

In [15]:
all_data.category_id.value_counts().shape

(705,)

In [ ]:
vc = train.brand_id.value_counts()
vc[vc<20].shape

# Model Learning

In [12]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk import word_tokenize

In [13]:
class Tokenizer(object):
    def __init__(self):
        self.tokenizer = word_tokenize
        self.stop_words = set(
        ['is', 'of', 'it', 'at', 'on', 'and', 'as', 'the', 'to', 'are', 'this', 'that', 'be', 'in',
          'an', 'or','any', 'all', 'am','you','we', '__NUMBER__', '__SERIAL__'])

    def __call__(self, text):
        text = text.lower()
        # replace special characters
        text = re.sub(r'[^a-z0-9\s/\\_\t,\-]', '', text,flags=re.IGNORECASE)
        text = re.sub(r'[/\\_\t,-]', ' ', text,flags=re.IGNORECASE)
        # replace numbers to reduce number of features
        text = re.sub(r'\b[0-9]+\b', ' __NUMBER__ ', text) 
        # replace possible product/serial numbers
        text = re.sub(r'\b\w*\d+\w*\d?\b', ' __SERIAL__ ', text)
                
        tokens = [w for w in self.tokenizer(text) if (w not in self.stop_words and len(w)>1)]
        # only return first and last two tokens
        return tokens if len(tokens) <5 else tokens[:2] + tokens[-2:]

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=Tokenizer())
print_log("starting vectorizer fit_transform")
sparse_title = vectorizer.fit_transform(train['product_title'])
print_log("completed vectorizer fit_transform")

In [ ]:
print("distinct words found", len(vectorizer.vocabulary_))

To build sparse matrix from vectorized tokens and category_id  
learn model on top of this

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
category_dict_vectorizer = DictVectorizer()
print_log("starting sparse category")
sparse_category = category_dict_vectorizer.fit_transform(train.category_id.astype(str).apply(lambda x: {x: 1}))
print_log("completed sparse category")

In [ ]:
sparse_category.shape, (train.category_id.shape, train.category_id.nunique())

In [ ]:
from scipy.sparse import hstack

In [ ]:
joined_data = hstack([sparse_category, sparse_title], format='csr')

In [ ]:
joined_data.shape

Saving file

In [ ]:
from scipy.io import mmwrite, mmread
mmwrite('joined_data.mtx', joined_data)
mmwrite('sparse_category.mtx', sparse_category)
mmwrite('sparse_title.mtx', sparse_title)

In [ ]:
# joined_data = mmread('joined_data.mtx')
# sparse_category = mmread('sparse_category')
# sparse_title = mmread('sparse_title')

Model validation
--

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.cross_validation import ShuffleSplit
from sklearn.naive_bayes import MultinomialNB

In [ ]:
import importlib
import util

importlib.reload(util)
from util import print_log, validate_model, sparse_validate_model

In [ ]:
# Validate logistic regression 
print_log("starting validation")
# clf = LogisticRegression()
# clf = SGDClassifier(loss='log')
clf = MultinomialNB()
util.sparse_validate_model(X=joined_data, Y=pd.np.ravel(train.brand_id.astype(int)), classifier=clf, 
               split_generator=lambda Y: ShuffleSplit(n=Y.shape[0], n_iter=3, test_size=0.4))
print_log("completed validation")

Apply Model
--

Data preparation

In [ ]:
test_title_sparse = sparse_title = vectorizer.transform(test['product_title'])
test_category_sparse = category_dict_vectorizer.transform(test.category_id.astype(str).apply(lambda x: {x: 1}))
joined_test_data = hstack([sparse_category, sparse_title], format='csr')

In [ ]:
mmwrite('joined_test_data.mtx', joined_test_data)
mmwrite('test_title_sparse', test_title_sparse)
mmwrite('test_category_sparse', test_category_sparse)

In [ ]:
joined_test_data = mmread('joined_test_data.mtx')
test_title_sparse = mmread('test_title_sparse.mtx')
test_category_sparse = mmread('test_category_sparse.mtx')

predict

In [ ]:
test_pred = clf.predict(joined_test_data)

# category wise model strategy

In [19]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from sklearn.externals import joblib

class Tokenizer(object):
    def __init__(self):
        self.tokenizer = word_tokenize
        self.stop_words = set(
        ['is', 'of', 'it', 'at', 'on', 'and', 'as', 'the', 'to', 'are', 'this', 'that', 'be', 'in',
          'an', 'or','any', 'all', 'am','you','we', '__NUMBER__', '__SERIAL__'])

    def __call__(self, text):
        text = text.lower()
        # replace special characters
        text = re.sub(r'[^a-z0-9\s/\\_\t,\-]', '', text,flags=re.IGNORECASE)
        text = re.sub(r'[/\\_\t,-]', ' ', text,flags=re.IGNORECASE)
        # replace numbers to reduce number of features
        text = re.sub(r'\b[0-9]+\b', ' __NUMBER__ ', text) 
        # replace possible product/serial numbers
        text = re.sub(r'\b\w*\d+\w*\d?\b', ' __SERIAL__ ', text)
                
        tokens = [w for w in self.tokenizer(text) if (w not in self.stop_words and len(w)>1)]
        # only return first and last two tokens
        return tokens if len(tokens) <5 else tokens[:3] + tokens[-2:]
    
def learn_model_for_category(train_df):
    category = train.category_id.iloc[0]
    learner = MultinomialNB()
    vectorizer = TfidfVectorizer(tokenizer=Tokenizer())
    estimators = [('transform', vectorizer), ('learner', learner)]
    pipe_line = Pipeline(estimators)
    pipe_line.fit(train_df['product_title'].values,train_df['brand_id'].astype(int))
    joblib.dump(pipe_line,'category_'+str(category)+'_model.clf')
    return True

In [20]:
category_size = train.category_id.value_counts()
major_cats = category_size[category_size>100].shape

In [21]:
%time cat_models = train.groupby('category_id').apply(learn_model_for_category)

CPU times: user 4min 24s, sys: 7.13 s, total: 4min 31s
Wall time: 5min 22s
